### Import required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from CartPole import CartPole
from CartPole_GPS import CartPole_GPS

from ilqr.dynamics import constrain
from copy import deepcopy

from EstimateDynamics import local_estimate
from GMM import Estimated_Dynamics_Prior

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

from mujoco_py import load_model_from_path, MjSim, MjViewer
import mujoco_py

import time

### Formulate the iLQR problem

In [2]:
'''
1 - dt = time step
2 - N = Number of control points in the trajectory
3 - x0 = Initial state
4 - x_goal = Final state
5 - Q = State cost
6 - R = Control cost
7 - Q_terminal = Cost at the final step
8 - x_dynamics array stores the information regarding system. 
    x_dynamics[0] = m = mass of the pendulum bob 
    x_dynamics[1] = M = mass of the cart 
    x_dynamics[2] = L = length of the massless rod 
    x_dynamics[3] = g = gravity 
    x_dynamics[4] = d = damping in the system
'''
dt = 0.005
N = 1000  # Number of time steps in trajectory.
x_dynamics = np.array([1, 5, 2, 9.80665, 1]) # m=1, M=5, L=0.3, g=9.80665, d=0.05
x0 = np.array([0.0, 0.0, 0.01, 0.0])  # Initial state
x_goal = np.array([1.0, 0.0, 0.0, 0.0])
# Instantenous state cost.
Q = np.eye(5)
Q[0,0] = 10
Q[2, 2] = 100
Q[3, 3] = 100
# Q[4, 4] = 100
# Terminal state cost.
Q_terminal = 100 * np.eye(5)
# Instantaneous control cost.
R = np.array([[1.0]])

### iLQR on Cart Pole

In [3]:
cartpole_prob = CartPole(dt, N, x_dynamics, x0, x_goal, Q, R, Q_terminal)
xs, us, K, k = cartpole_prob.run_IterLinQuadReg()

iteration 0 failed 94157.13325327754 [ 0.05813849 -0.13913414 -0.0227728   0.49281513]
iteration 1 failed 94157.13325327754 [ 0.05813849 -0.13913414 -0.0227728   0.49281513]
iteration 2 accepted 89846.56045705981 [1.06773649 0.04699051 0.2020933  0.93437923]
iteration 3 accepted 89567.0303410186 [1.31765305 0.13159595 0.02437988 0.82907033]
iteration 4 accepted 89422.26173618884 [1.07632835 0.02766959 0.20785807 1.06497194]
iteration 5 accepted 89175.34785541099 [1.08738083 0.02578999 0.13556291 1.01263584]
iteration 6 accepted 89000.93251305995 [1.10958017 0.01550756 0.07496078 1.0354927 ]
iteration 7 accepted 88782.44136765118 [ 1.10166213e+00  3.26721171e-04 -1.40411317e-02  9.88393083e-01]
iteration 8 accepted 88677.45287893196 [ 1.22185282e+00  2.20315458e-04 -2.03546407e-01  1.05903102e+00]
iteration 9 accepted 88671.7543721352 [ 1.21297272e+00 -8.01997727e-04 -1.88533079e-01  1.05591341e+00]
iteration 10 accepted 88670.38586931361 [ 1.20583742 -0.00128378 -0.17264522  1.05297322

In [4]:
# State matrix split into individual states. For plotting and analysing purposes.
t = np.arange(N + 1) * dt
x = xs[:, 0] # Position
x_dot = xs[:, 1] # Velocity
theta = np.unwrap(cartpole_prob.deaugment_state(xs)[:, 2])  # Theta, makes for smoother plots.
theta_dot = xs[:, 3] # Angular velocity

### Simulate the real system and generate the data
Cost matrices, initial position and goal position will remain same as the above problem. As it indicates one policy. But still the initial positions and goal positions must be passed explicitly to the function. But you don't need to pass cost matrices (assume penalty on the system is same), this is just used to use to calculate the cost of the trajectory. Correct control action must be passed. Parameter gamma indicates how much of original data you want to keep

Variance of the Gaussian noise will be taken as input from a Unif(0, var_range) uniform distribution. Inputs: x_initial, x_goal, u, n_rollouts, pattern='Normal', pattern_rand=False, var_range=10, gamma=0.2, percent=20

Pattern controls how the control sequence will be modified after applying white Guassian noise (zero mean).
- Normal: based on the correction/mixing parameter gamma generate control (gamma controls how much noise we want).
- MissingValue: based on the given percentage, set those many values to zero (it is implicitly it uses "Normal" generated control is used). 
- Shuffle: shuffles the entire "Normal" generated control sequence.
- TimeDelay: takes the "Normal" generated control and shifts it by 1 index i.e. one unit time delay.
- Extreme: sets gamma as zeros and generates control based on only noise.

If 'pattern_rand' is 'True' then we don't need to send the explicitly, it will chose one randomly for every rollout (default is 'False'). If you want to chose specific pattern then send it explicitly. 

In [5]:
x_rollout, u_rollout, local_policy, cost = cartpole_prob.gen_rollouts(x0, x_goal, us, n_rollouts=10, pattern_rand=True, var_range=10, gamma=0.2, percent=20)

### Local system dynamics/model estimate
loca_estimate: function takes the states (arranged in a special format, [x(t), u(t), x(t+1)]), no. of gaussian mixtures and no.of states.

In [6]:
model = Estimated_Dynamics_Prior(init_sequential=False, eigreg=False, warmstart=True, 
                 min_samples_per_cluster=20, max_clusters=50, max_samples=20, strength=1.0)
model.update_prior(x_rollout, u_rollout)
A, B, C = model.fit(x_rollout, u_rollout)

In [7]:
print(A.shape)
print(B.shape)
print(C.shape)

(1001, 5, 6)
(1001, 5)
(1001, 5, 5)


In [8]:
Model = "mujoco/cartpole.xml"
model_loaded = load_model_from_path(Model)
sim = MjSim(model_loaded)

In [9]:
#viewer = mujoco_py.MjViewer(sim)
t = 0
sim.data.qpos[0] = 0.0
sim.data.qpos[1] = 0.01
sim.data.qvel[0] = 0
sim.data.qvel[1] = 0
final = 0
for i in range(1000):
    start_time = time.time()
    state = np.c_[sim.data.qpos[0],sim.data.qvel[0],np.sin(sim.data.qpos[1]),
                  np.cos(sim.data.qpos[1]),sim.data.qvel[1]].T
    control = np.dot(k[i,:],(xs[i].reshape(5,1) - state))  + K[i].T + us[i]
    sim.data.ctrl[0] = control
    sim.step()
    #viewer.render()
    print(control, "this is the control")
    print(sim.get_state())
    if (sim.data.qpos[0] == 1.0 and sim.data.qpos[1] == 0):
        print('states reached')
        break

[[-2.15463325]] this is the control
MjSimState(time=0.01, qpos=array([-0.0009475,  0.0122851]), qvel=array([-0.09475036,  0.2285099 ]), act=None, udd_state={})
[[-23.57716319]] this is the control
MjSimState(time=0.02, qpos=array([-0.0028422 ,  0.01685075]), qvel=array([-0.1894695 ,  0.45656548]), act=None, udd_state={})
[[-45.39113344]] this is the control
MjSimState(time=0.03, qpos=array([-0.00568405,  0.02369836]), qvel=array([-0.28418552,  0.68476029]), act=None, udd_state={})
[[-67.65385342]] this is the control
MjSimState(time=0.04, qpos=array([-0.0094733 ,  0.03283509]), qvel=array([-0.37892414,  0.91367298]), act=None, udd_state={})
[[-90.42235027]] this is the control
MjSimState(time=0.05, qpos=array([-0.01421037,  0.04427371]), qvel=array([-0.47370755,  1.14386214]), act=None, udd_state={})
[[-113.7527306]] this is the control
MjSimState(time=0.060000000000000005, qpos=array([-0.0198959 ,  0.05803231]), qvel=array([-0.5685531 ,  1.37586056]), act=None, udd_state={})
[[-137.69

MjSimState(time=2.5499999999999896, qpos=array([2.36455823, 1.9928961 ]), qvel=array([-1.69177048,  1.03894989]), act=None, udd_state={})
[[-259.13391]] this is the control
MjSimState(time=2.5599999999999894, qpos=array([2.34684048, 2.00459808]), qvel=array([-1.77177535,  1.17019821]), act=None, udd_state={})
[[-269.85581176]] this is the control
MjSimState(time=2.569999999999989, qpos=array([2.32832398, 2.01757693]), qvel=array([-1.85164964,  1.29788484]), act=None, udd_state={})
[[-280.12082748]] this is the control
MjSimState(time=2.579999999999989, qpos=array([2.3090101 , 2.03179362]), qvel=array([-1.9313883 ,  1.42166936]), act=None, udd_state={})
[[-289.89176379]] this is the control
MjSimState(time=2.5899999999999888, qpos=array([2.2889002 , 2.04720577]), qvel=array([-2.01099003,  1.54121502]), act=None, udd_state={})
[[-299.13171066]] this is the control
MjSimState(time=2.5999999999999885, qpos=array([2.26799562, 2.06376766]), qvel=array([-2.09045743,  1.65618905]), act=None, u

[[-0.85678746]] this is the control
MjSimState(time=3.1599999999999766, qpos=array([-0.16858815,  2.87637001]), qvel=array([-6.10169422, -0.09812883]), act=None, udd_state={})
[[10.5053877]] this is the control
MjSimState(time=3.1699999999999764, qpos=array([-0.22862837,  2.87825647]), qvel=array([-6.00402188,  0.18864676]), act=None, udd_state={})
[[-16.58658537]] this is the control
MjSimState(time=3.179999999999976, qpos=array([-0.28956779,  2.87867252]), qvel=array([-6.09394141,  0.04160463]), act=None, udd_state={})
[[-1.88247084]] this is the control
MjSimState(time=3.189999999999976, qpos=array([-0.35140612,  2.87762347]), qvel=array([-6.18383295, -0.10490529]), act=None, udd_state={})
[[12.51105009]] this is the control
MjSimState(time=3.1999999999999758, qpos=array([-0.41226769,  2.87944045]), qvel=array([-6.0861572 ,  0.18169824]), act=None, udd_state={})
[[-14.5676898]] this is the control
MjSimState(time=3.2099999999999755, qpos=array([-0.47402857,  2.87978368]), qvel=array

[[0.84597914]] this is the control
MjSimState(time=5.969999999999917, qpos=array([ 0.05724904, 19.42413231]), qvel=array([ 6.00592088, -0.3129395 ]), act=None, udd_state={})
[[1.00758365]] this is the control
MjSimState(time=5.979999999999917, qpos=array([ 0.11815396, 19.42059343]), qvel=array([ 6.09049194, -0.35388792]), act=None, udd_state={})
[[0.81230122]] this is the control
MjSimState(time=5.989999999999917, qpos=array([ 0.17973422, 19.41697802]), qvel=array([ 6.15802633, -0.36154111]), act=None, udd_state={})
[[1.02508608]] this is the control
MjSimState(time=5.9999999999999165, qpos=array([ 0.24216116, 19.41293122]), qvel=array([ 6.24269349, -0.40467981]), act=None, udd_state={})
[[0.82951497]] this is the control
MjSimState(time=6.009999999999916, qpos=array([ 0.30528002, 19.40875468]), qvel=array([ 6.31188594, -0.41765421]), act=None, udd_state={})
[[0.97836034]] this is the control
MjSimState(time=6.019999999999916, qpos=array([ 0.36922697, 19.40416141]), qvel=array([ 6.3946

[[23.1732625]] this is the control
MjSimState(time=6.619999999999903, qpos=array([ 2.50632008, 24.03831157]), qvel=array([-1.07241213, 15.09496985]), act=None, udd_state={})
[[21.13314353]] this is the control
MjSimState(time=6.629999999999903, qpos=array([ 2.49547622, 24.18671074]), qvel=array([-1.08438563, 14.83991671]), act=None, udd_state={})
[[19.48536118]] this is the control
MjSimState(time=6.639999999999903, qpos=array([ 2.4846469 , 24.33259981]), qvel=array([-1.0829327 , 14.58890749]), act=None, udd_state={})
[[18.21324587]] this is the control
MjSimState(time=6.649999999999903, qpos=array([ 2.47397621, 24.47596456]), qvel=array([-1.0670683 , 14.33647455]), act=None, udd_state={})
[[17.29324593]] this is the control
MjSimState(time=6.659999999999902, qpos=array([ 2.46361461, 24.61674482]), qvel=array([-1.03616084, 14.07802606]), act=None, udd_state={})
[[16.69740345]] this is the control
MjSimState(time=6.669999999999902, qpos=array([ 2.4537145 , 24.75485134]), qvel=array([-0.

[[-65.46244545]] this is the control
MjSimState(time=9.469999999999843, qpos=array([ 0.28304737, 39.7753632 ]), qvel=array([-6.04157013,  4.11812547]), act=None, udd_state={})
[[-67.64519202]] this is the control
MjSimState(time=9.479999999999842, qpos=array([ 0.22190287, 39.81758715]), qvel=array([-6.11445037,  4.22239549]), act=None, udd_state={})
[[-69.90298367]] this is the control
MjSimState(time=9.489999999999842, qpos=array([ 0.16003041, 39.86073752]), qvel=array([-6.18724568,  4.31503702]), act=None, udd_state={})
[[-72.23177434]] this is the control
MjSimState(time=9.499999999999842, qpos=array([ 0.09742956, 39.90469242]), qvel=array([-6.26008495,  4.39549013]), act=None, udd_state={})
[[-74.6274425]] this is the control
MjSimState(time=9.509999999999842, qpos=array([3.40985166e-02, 3.99493244e+01]), qvel=array([-6.33310465,  4.46319645]), act=None, udd_state={})
[[-77.08579374]] this is the control
MjSimState(time=9.519999999999841, qpos=array([-2.99659469e-02,  3.99945005e+0

In [10]:
from Simulator import Mujoco_sim
cart_pole_simulator = Mujoco_sim(Model,True)
cart_pole_simulator.load(xs,us,k,K,x0,initial=False)

model added to the simulator


In [11]:
cart_pole_simulator.runSimulation()

Creating window glfw


In [ ]:
np.append(np.expand_dims(state,1),np.expand_dims(np.append(sim.data.qpos,sim.data.qvel),1),axis = 1)

In [ ]:
k.shape